In [1]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
import torch
import numpy as np
import os

In [3]:
pathReq=os.getcwd()+'/'
stats=np.load(pathReq+'allStats20210513.npy')

forInference=np.load(pathReq+'statsForInference.npy')
stats.shape,forInference.shape

((3230, 30), (47, 29))

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

h=['PA','AB','R','H','2B','3B','HR',
           'RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS']#batting only
hBvsP=['PA2','AB2','H2', '1B2','2B2','3B2','HR2','BB2','SO2','AVG2','OBP2','SLG2'] #BvsP
hTotal=h+hBvsP

skipping=0#len(h) #sometimes we might just want to do batter vs pitcher so we skip batter stats
features=torch.tensor(stats[:,skipping:-1])
labels=torch.tensor(stats[:,-1])

forInferenceStats=torch.tensor(forInference[:,skipping:])

pctTotal=(1-torch.mean(labels))*2
finalNo=(features.size()[0]*pctTotal).int()

balancedFeatures=torch.zeros(finalNo,features.size()[1])
balancedLabels=torch.zeros(finalNo)

for smpl, l in enumerate(labels):
  if l==1 and smpl<(finalNo//2):
    balancedFeatures[smpl]=features[smpl]
    balancedLabels[smpl]=l
  elif l==0 and smpl<finalNo:
    balancedFeatures[smpl]=features[smpl]
    balancedLabels[smpl]=l

X_train, X_test, y_train, y_test = train_test_split(balancedFeatures, balancedLabels, test_size=0.1, random_state=42)

scaler = StandardScaler()

x_norm=torch.tensor(scaler.fit_transform(X_train))

x_norm_test=torch.tensor(scaler.fit_transform(X_test))

inference_norm=torch.tensor(scaler.fit_transform(forInferenceStats))


In [6]:
x_norm.shape,x_norm_test.shape,inference_norm.shape

(torch.Size([1900, 29]), torch.Size([212, 29]), torch.Size([47, 29]))

In [23]:
from torch import nn
device='cpu'
net = nn.Sequential(
      nn.Linear(features.shape[1], 30),
      nn.ReLU(),           
      nn.Linear(30, 20),                                 
      nn.ReLU(),
      nn.Linear(20, 10),
      nn.ReLU(),
      nn.Linear(10, 1),
      nn.Sigmoid(),
      ).to(device)

net=net.double()

PATH=pathReq + '/savedModels/overlyConfident2021-05-18.pth'
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [24]:
# net[-4:]

In [ ]:
from torch import nn
import torch.optim as optim

netHold=net[:-4]
netTrain=net[-4:]
netHold.eval()
netTrain.train()

device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')

netHold=netHold.double()
netTrain=netTrain.double()

losses=[]

correct=0
count=0

batchSize=1

epochs=100
optimizer = optim.Adam(netTrain.parameters(), lr=0.1, weight_decay=1e-5)

for epoch in range(epochs):
    for sample, label in zip(x_norm, y_train):
        sample=torch.tensor(sample).to(device)
        label=torch.tensor(label).to(device)
        sample=sample.double()
        label=label.double()
        
        outputHold=netHold(sample)        
        output=netTrain(outputHold)        
        netTrain.zero_grad()
        
        target = label
        target = target.view(1)  # make it the same shape as output 
        criterion = nn.BCELoss()

        # in your training loop:
        optimizer.zero_grad()   # zero the gradient buffers

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()    # Does the update
        losses.append(loss)
        count+=batchSize
        if (output>0.5 and label==1) or (output<=0.5 and label==0):
              correct+=1
                
    acc=correct/count
    
    print('epoch=' ,epoch, 'avg loss=', torch.mean(torch.tensor(losses)), 'acc = ', acc)


/home/adimn/anaconda3/envs/venv-base/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/adimn/anaconda3/envs/venv-base/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


epoch= 0 avg loss= tensor(1.4466, dtype=torch.float64) acc =  0.8410526315789474
epoch= 1 avg loss= tensor(1.1878, dtype=torch.float64) acc =  0.8305263157894737
epoch= 2 avg loss= tensor(1.0666, dtype=torch.float64) acc =  0.842280701754386
epoch= 3 avg loss= tensor(1.0539, dtype=torch.float64) acc =  0.8455263157894737
epoch= 4 avg loss= tensor(4.3474, dtype=torch.float64) acc =  0.8174736842105264
epoch= 5 avg loss= tensor(3.8349, dtype=torch.float64) acc =  0.8106140350877193
epoch= 6 avg loss= tensor(3.7457, dtype=torch.float64) acc =  0.8098496240601504
epoch= 7 avg loss= tensor(3.4980, dtype=torch.float64) acc =  0.8095394736842105
epoch= 8 avg loss= tensor(3.3132, dtype=torch.float64) acc =  0.8042690058479532
epoch= 9 avg loss= tensor(3.1463, dtype=torch.float64) acc =  0.8028421052631579
epoch= 10 avg loss= tensor(2.9816, dtype=torch.float64) acc =  0.801244019138756
epoch= 11 avg loss= tensor(5.3700, dtype=torch.float64) acc =  0.7764035087719299
epoch= 12 avg loss= tensor(6

In [21]:
from torch import nn
import torch.optim as optim

device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')

#20,10,5 (40%)
#30,20,10 (58%)(overfit)

# net = nn.Sequential(
#       nn.Linear(features.shape[1], 10),
#       nn.ReLU(),           
#       nn.Linear(10, 5),                 
#       nn.ReLU(),
#       nn.Linear(5, 5),                 
#       nn.ReLU(),
#       nn.Linear(5, 1),
#       nn.Sigmoid(),
#       ).to(device)

net=net.double()

losses=[]

correct=0
count=0
# X_train.to(device)
# y_train.to(device)

# X_test.to(device)
# y_test.to(device)
def train_gen(feat,lbl, batch=4):
  fBatch=[]#torch.zeros(batch, feat.size()[1])
  lBatch=[]#torch.zeros(lbl)
  for i, f in enumerate(feat):   
    fBatch.append(f.numpy())
    lBatch.append(lbl[i].numpy())
    if len(fBatch)==batch:
      yield fBatch, lBatch
      fBatch,lBatch=[],[]
  
batchSize=1

epochs=100

for epoch in range(epochs):
    gen=train_gen(X_train,y_train,batchSize)
    for sample, label in zip(x_norm, y_train):
    #for sample, label in gen:
        sample=torch.tensor(sample).to(device)
        label=torch.tensor(label).to(device)
        #label=torch.from_numpy(np.array(label)).to(device)
        sample=sample.double()
        label=label.double()
        
        output=net(sample)        
        net.zero_grad()
        target = label
        #target = target.view(-1, 1)  # make it the same shape as output #this is for batches
        target = target.view(1)  # make it the same shape as output 
        #print('pred:',output, 'pred shape:',output.shape, 'label:', target, 'label shape:', target.shape)
        criterion = nn.BCELoss()

        optimizer = optim.Adam(net.parameters(), lr=0.01)

        # in your training loop:
        optimizer.zero_grad()   # zero the gradient buffers

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()    # Does the update
        losses.append(loss)
        count+=batchSize
        if (output>0.5 and label==1) or (output<=0.5 and label==0):
        #for i, o in enumerate(output):
        #  print('o', o>0.5, 'i', label[i]>0.5)
        #  if (o>0.5 and label[i]==1) or (o<=0.5 and label[i]==0):
              correct+=1
#         print('output=', output.item(), ' label = ', label.item())
    acc=correct/count
    
    print('epoch=' ,epoch, 'avg loss=', torch.mean(torch.tensor(losses)), 'acc = ', acc)


/home/adimn/anaconda3/envs/venv-base/lib/python3.7/site-packages/ipykernel_launcher.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/adimn/anaconda3/envs/venv-base/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


epoch= 0 avg loss= tensor(0.5172, dtype=torch.float64) acc =  0.9131578947368421
epoch= 1 avg loss= tensor(0.6248, dtype=torch.float64) acc =  0.9194736842105263
epoch= 2 avg loss= tensor(0.7431, dtype=torch.float64) acc =  0.9235087719298246
epoch= 3 avg loss= tensor(0.8404, dtype=torch.float64) acc =  0.9260526315789473
epoch= 4 avg loss= tensor(0.9263, dtype=torch.float64) acc =  0.9275789473684211


KeyboardInterrupt: 

In [7]:
test=np.load(pathReq+'allStats20210513.npy')
testfeats=test[:,:-1]

x_norm_test2=torch.tensor(scaler.fit_transform(testfeats))
y_test2=test[:,-1]

In [39]:
from datetime import datetime
dateReq=datetime.today().strftime('%Y-%m-%d')
PATH=pathReq + '/savedModels/overlyConfident'+dateReq+'.pth'

# for param_tensor in net.state_dict():
#     print(param_tensor, "\t", net.state_dict()[param_tensor])
torch.save(net.state_dict(), PATH)
net.load_state_dict(torch.load(PATH))
net.eval()

Sequential(
  (0): Linear(in_features=29, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=5, bias=True)
  (3): ReLU()
  (4): Linear(in_features=5, out_features=5, bias=True)
  (5): ReLU()
  (6): Linear(in_features=5, out_features=1, bias=True)
  (7): Sigmoid()
)

In [32]:
net.eval()
net.cpu()
indices=500
net(x_norm[:indices].cpu()),y_train[:indices],net(x_norm_test[:indices].cpu()),y_test[:indices]

# for i in range(indices):
#   print('pred=', net(x_norm[i].cpu()).item(),'label=', y_train[i])

for i in range(indices):
  pred_score=net(x_norm_test2[i].cpu()).item()
  if pred_score >0.5:
    print('pred test=', pred_score ,'label=', y_test2[i])

pred test= 0.9999999998810889 label= 1.0
pred test= 0.9999999956599717 label= 0.0
pred test= 0.9999999956599717 label= 1.0
pred test= 0.9999998912015452 label= 1.0
pred test= 0.9999999956599717 label= 0.0
pred test= 0.9999999956599717 label= 1.0
pred test= 0.9999999956599717 label= 0.0
pred test= 0.9999999957516867 label= 1.0
pred test= 0.9999999956599717 label= 1.0
pred test= 0.9999999956599717 label= 1.0
pred test= 0.9999999804150467 label= 1.0
pred test= 0.9999999956599717 label= 1.0
pred test= 0.9467277134969712 label= 1.0
pred test= 0.9999999975551932 label= 1.0
pred test= 0.9999999956599717 label= 1.0
pred test= 0.9999999956599717 label= 1.0
pred test= 0.9999999956599717 label= 1.0
pred test= 0.9999999956599717 label= 1.0
pred test= 0.9999999956599717 label= 1.0
pred test= 0.9999999956599717 label= 1.0
pred test= 0.9999999999636351 label= 0.0
pred test= 0.9999999956599717 label= 1.0
pred test= 0.9999995896238477 label= 1.0
pred test= 0.99999999999938 label= 1.0
pred test= 0.99999

In [33]:
net.eval()
net.cpu()
preds=net(inference_norm)

In [34]:
inference_norm.shape

torch.Size([48, 29])

In [35]:

preds.shape

torch.Size([48, 1])

In [36]:
pList=[['Rafael Devers', 'Boston'],
 ['Willie Calhoun', 'Texas'],
 ['Eric Hosmer', 'San Diego'],
 ['Rhys Hoskins', 'Philadelphia'],
 ['Bryce Harper', 'Philadelphia'],
 ['Jesus Aguilar', 'Miami'],
 ['Trevor Story', 'Colorado'],
 ['Nick Castellanos', 'Cincinnati'],
 ['Ryan McMahon', 'Colorado'],
 ['Justin Turner', 'Los Angeles'],
 ['DJ LeMahieu', 'New York'],
 ['Alex Bregman', 'Houston'],
 ['Miguel Rojas', 'Miami'],
 ['Paul Goldschmidt', 'St. Louis'],
 ['Yoan Moncada', 'Chicago'],
 ['Trent Grisham', 'San Diego'],
 ['Yonathan Daza', 'Colorado'],
 ['Josh Rojas', 'Arizona'],
 ['Carlos Santana', 'Kansas City'],
 ['Yuli Gurriel', 'Houston'],
 ['David Fletcher', 'Los Angeles'],
 ['Dylan Carlson', 'St. Louis'],
 ['Raimel Tapia', 'Colorado'],
 ['Austin Riley', 'Atlanta'],
 ['David Peralta', 'Arizona'],
 ['Marcus Semien', 'Toronto'],
 ['Salvador Perez', 'Kansas City'],
 ['Andrew Benintendi', 'Kansas City'],
 ['Alex Verdugo', 'Boston'],
 ['Cedric Mullins', 'Baltimore'],
 ['Kris Bryant', 'Chicago'],
 ['Jesse Winker', 'Cincinnati'],
 ['Xander Bogaerts', 'Boston'],
 ['Michael Brantley', 'Houston'],
 ['Tommy Edman', 'St. Louis'],
 ['Giancarlo Stanton', 'New York'],
 ['Jared Walsh', 'Los Angeles'],
 ['J.D. Martinez', 'Boston'],
 ['Starlin Castro', 'Washington'],
 ['Nolan Arenado', 'St. Louis'],
 ['Nate Lowe', 'Texas'],
 ['Tim Anderson', 'Chicago'],
 ['Trea Turner', 'Washington'],
 ['Isiah Kiner-Falefa', 'Texas'],
 ['Adam Frazier', 'Pittsburgh'],
 ['Pavin Smith', 'Arizona'],
 ['Nick Solak', 'Texas'],
 ['Trey Mancini', 'Baltimore']]



In [38]:
from datetime import datetime
print(datetime.today().strftime('%Y-%m-%d'))
nnOut=[]
for i, ind in enumerate(torch.topk(preds.reshape(1,preds.shape[0]),20)[1][0]):  
  nnOut.append(pList[ind])
  print(pList[ind],torch.topk(preds.reshape(1,preds.shape[0]),20)[0][0][i].item())

2021-05-15
['David Fletcher', 'Los Angeles'] 1.0
['Pavin Smith', 'Arizona'] 1.0
['Yonathan Daza', 'Colorado'] 1.0
['Ryan McMahon', 'Colorado'] 0.9999999999999991
['Dylan Carlson', 'St. Louis'] 0.9999999999972427
['Nick Solak', 'Texas'] 0.9999999999873859
['Nolan Arenado', 'St. Louis'] 0.9999999999864697
['Tommy Edman', 'St. Louis'] 0.999999999022096
['Willie Calhoun', 'Texas'] 0.99999999819039
['Nate Lowe', 'Texas'] 0.9999999978206273
['Cedric Mullins', 'Baltimore'] 0.9999999956599717
['Tim Anderson', 'Chicago'] 0.9999999956599717
['Raimel Tapia', 'Colorado'] 0.9999999956599717
['Salvador Perez', 'Kansas City'] 0.9999999956599717
['Michael Brantley', 'Houston'] 0.9999999956599717
['J.D. Martinez', 'Boston'] 0.9999999956599717
['Trey Mancini', 'Baltimore'] 0.9999999956599717
['Starlin Castro', 'Washington'] 0.9999999956599717
['Xander Bogaerts', 'Boston'] 0.9999999956599717
['Jesse Winker', 'Cincinnati'] 0.9999999956599717


In [ ]:
len(pList)

49

In [22]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0, max_iter=1000).fit(x_norm, y_train)

In [23]:
clf.score(x_norm_test, y_test)

0.8152173913043478

In [24]:
from datetime import datetime
print(datetime.today().strftime('%Y-%m-%d'))

lrOut=[]
for i, pred in enumerate(clf.predict_proba(inference_norm)[:,1][np.argsort(-clf.predict_proba(inference_norm)[:,1])][:10]):  
  lrOut.append(pList[i])
  print(pList[i], pred)

2021-05-15
['Paul Goldschmidt', 'St. Louis'] 0.999780017203643
['Rafael Devers', 'Boston'] 0.9848612150580556
['Randal Grichuk', 'Toronto'] 0.9346816196472018
['Giancarlo Stanton', 'New York'] 0.9259634913698378
['Andrew Benintendi', 'Kansas City'] 0.8824150199235107
['Jesus Aguilar', 'Miami'] 0.8433218626593961
['Shohei Ohtani', 'Los Angeles'] 0.8396202610265788
['Trey Mancini', 'Baltimore'] 0.8273806263552075
['Rhys Hoskins', 'Philadelphia'] 0.7735909562856644
['Corey Seager', 'Los Angeles'] 0.7596874128563706


In [27]:
for o in nnOut:
  if o in lrOut:
    print(o)

['Paul Goldschmidt', 'St. Louis']
['Corey Seager', 'Los Angeles']
['Randal Grichuk', 'Toronto']
['Giancarlo Stanton', 'New York']
['Rhys Hoskins', 'Philadelphia']
['Rafael Devers', 'Boston']
['Jesus Aguilar', 'Miami']


In [ ]:
# 2021-05-12
# ['Jake Cronenworth', 'San Diego'] 0.9951982878094792
# ['Paul Goldschmidt', 'St. Louis'] 0.9885262239419107
# ['Yermin Mercedes', 'Chicago'] 0.9743571206807747
# ['Carlos Santana', 'Kansas City'] 0.9727445792429186
# ['Mike Trout', 'Los Angeles'] 0.9636184834153763
# ['Austin Riley', 'Atlanta'] 0.9586897621663748
# ['Randal Grichuk', 'Toronto'] 0.9498869962253059
# ['J.D. Martinez', 'Boston'] 0.9106846220235486
# ['Trey Mancini', 'Baltimore'] 0.8654366296051179
# ['Trent Grisham', 'San Diego'] 0.7794279905260642



In [ ]:
np.argsort(-clf.predict_proba(inference_norm)[:,1]),clf.predict_proba(inference_norm)[:,1],clf.predict_proba(inference_norm)[:,1][np.argsort(-clf.predict_proba(inference_norm)[:,1])]


(array([18,  9, 11, 32, 34, 28, 36, 29, 15,  4, 39,  8,  3, 35,  0, 19, 26,
        33,  2, 13, 21, 42,  6, 14, 30, 38, 41, 40, 37, 31,  7, 25, 17, 10,
        46,  1, 43, 24, 22, 45, 20, 16, 12,  5, 44, 27, 23]),
 array([0.45747394, 0.07671667, 0.42466979, 0.50921759, 0.64018901,
        0.0657057 , 0.26373826, 0.08482591, 0.56207819, 0.8352786 ,
        0.0806784 , 0.83372114, 0.0657057 , 0.37323229, 0.25208786,
        0.66286895, 0.0657057 , 0.08482591, 0.9721208 , 0.44813441,
        0.0657057 , 0.36740579, 0.06914139, 0.0657057 , 0.06914139,
        0.08482591, 0.43756551, 0.0657057 , 0.68468595, 0.66286895,
        0.21396768, 0.08939631, 0.82797847, 0.42979197, 0.71715111,
        0.47543885, 0.66309993, 0.10858724, 0.19750926, 0.62796537,
        0.13930828, 0.18658975, 0.35013712, 0.07651625, 0.0657057 ,
        0.0657057 , 0.07671667]),
 array([0.9721208 , 0.8352786 , 0.83372114, 0.82797847, 0.71715111,
        0.68468595, 0.66309993, 0.66286895, 0.66286895, 0.64018901,
    

In [ ]:
h=['PA','AB','R','H','2B','3B','HR',
           'RBI','BB','IBB','SO','HBP','SH','SF','GDP','SB','CS']#batting only
hBvsP=['PA2','AB2','H2', '1B2','2B2','3B2','HR2','BB2','SO2','AVG2','OBP2','SLG2'] #BvsP

clf.coef_

array([[ 0.15407444,  0.12765536, -0.51097697, -1.28875385, -0.11525518,
        -0.02817996, -0.53926629,  0.40723739,  0.94032604,  0.08294364,
         1.06828656,  0.04501935,  0.        ,  0.53586396,  0.33600777,
        -0.05340233,  0.14588743,  0.59336036]])

In [ ]:
h[8]

'BB'

In [ ]:
y_test[:15]

tensor([1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1.])

In [ ]:
pctTotal=(1-torch.mean(labels))*2

finalNo=(features.size()[0]*pctTotal).int()

balancedSet=torch.zeros(finalNo,features.size()[1])

for smpl, l in enumerate(labels):
  if l==1 and smpl<(finalNo/2):
    balancedSet[smpl]=features[smpl].copy()
  else:
    balancedSet[smpl]=features[smpl].copy()



IndexError: ignored

In [ ]:
finalNo

tensor(914, dtype=torch.int32)